In [ ]:
import sys
sys.path.insert(0, '..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import date

from src.models import Option
from src.pricing import bs_call_price, bs_greeks, implied_vol
from src.backtester import DeltaHedger, PnLAttribution, PositionManager
from src.utils import load_price_data

In [ ]:
prices = load_price_data('../data/sample_prices.csv')
prices.head()

In [ ]:
opt = Option(
    strike=475,
    expiry=date(2024, 2, 16),
    option_type='call'
)

print(f'Strike: {opt.K}')
print(f'Expiry: {opt.expiry}')
print(f'Time to expiry: {opt.time_to_expiry(date(2024, 1, 2)):.3f} years')

In [ ]:
S = 472.65
K = 475
T = opt.time_to_expiry(date(2024, 1, 2))
r = 0.05
sigma = 0.15

price = bs_call_price(S, K, T, r, sigma)
greeks = bs_greeks(S, K, T, r, sigma, 'call')

print(f'Option price: ${price:.2f}')
print(f'Delta: {greeks["delta"]:.4f}')
print(f'Gamma: {greeks["gamma"]:.4f}')
print(f'Theta: {greeks["theta"]:.4f}')
print(f'Vega: {greeks["vega"]:.4f}')

In [ ]:
# run delta hedge simulation
hedger = DeltaHedger(
    option=opt,
    spot_data=prices,
    constant_vol=0.15,
    r=0.05,
    rehedge_frequency='daily',
    transaction_cost=0.01,
)

result = hedger.run()

In [ ]:
hedger.summary()

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 8))

axes[0, 0].plot(result['date'], result['spot'])
axes[0, 0].set_title('Spot Price')
axes[0, 0].set_ylabel('Price')

axes[0, 1].plot(result['date'], result['delta'])
axes[0, 1].set_title('Option Delta')
axes[0, 1].set_ylabel('Delta')

axes[1, 0].plot(result['date'], result['hedge_shares'])
axes[1, 0].set_title('Hedge Position (shares)')
axes[1, 0].set_ylabel('Shares')

axes[1, 1].plot(result['date'], result['cumulative_pnl'])
axes[1, 1].set_title('Cumulative PnL')
axes[1, 1].set_ylabel('PnL ($)')

for ax in axes.flat:
    ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# P&L attribution
from src.backtester import attribute_hedged_pnl

attr_df = attribute_hedged_pnl(result)
attr_df.head()

In [ ]:
# summarize attribution
print(f"Delta PnL: ${attr_df['delta_pnl'].sum():.2f}")
print(f"Gamma PnL: ${attr_df['gamma_pnl'].sum():.2f}")
print(f"Theta PnL: ${attr_df['theta_pnl'].sum():.2f}")
print(f"Vega PnL: ${attr_df['vega_pnl'].sum():.2f}")
print(f"Residual: ${attr_df['residual'].sum():.2f}")
print(f"Total: ${attr_df['total'].sum():.2f}")

In [ ]:
# position manager example
mgr = PositionManager()

# add a call and a put
call = Option(strike=475, expiry=date(2024, 2, 16), option_type='call')
put = Option(strike=470, expiry=date(2024, 2, 16), option_type='put')

mgr.add_position(call, quantity=1, entry_price=5.0, entry_date=date(2024, 1, 2), entry_spot=472.65, tag='long_call')
mgr.add_position(put, quantity=-1, entry_price=4.0, entry_date=date(2024, 1, 2), entry_spot=472.65, tag='short_put')

mgr.to_dataframe(spot=480, as_of=date(2024, 1, 15), constant_vol=0.15)

In [ ]:
# portfolio greeks
mgr.get_portfolio_greeks(spot=480, as_of=date(2024, 1, 15), constant_vol=0.15)